# Georeferencing

This notebook shows how to add Cartesian (x, y, z) and geographic (lat, lon, alt) coordinates to radar data using xradar.


- **DataTree** refers to [xarray.DataTree](https://docs.xarray.dev/en/stable/generated/xarray.DataTree.html#xarray.DataTree)
- **Dataset** refers to [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset)
- **DataArray** refers to [xarray.DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray)

In [ ]:
import xradar as xd
import xarray as xr
import cmweather  # noqa
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from open_radar_data import DATASETS
import warnings

warnings.simplefilter("ignore")

## Read Data

In [ ]:
file = DATASETS.fetch("sample_rainbow_5_59.vol")

In [ ]:
radar = xd.io.open_rainbow_datatree(file)

## Georeferencing (Cartesian)

In [ ]:
radar = radar.xradar.georeference()

In [ ]:
display(radar["sweep_0"].ds)

- **Notice that the radar DataTree now includes x, y, and z coordinates.**
- **Next, let’s add lat, lon, and alt coordinates.**

In [ ]:
del radar

## Georeferencing (Geographic)

In [ ]:
# Reading data again
radar = xd.io.open_rainbow_datatree(file)

In [ ]:
radar = radar.xradar.georeference(geo=True)
display(radar["sweep_0"].ds)

- **When geo=True, lat, lon, and alt coordinates are added instead.**

In [ ]:
del radar

## Other Uses

- **You can also add these coordinates to a single DataArray or Dataset.**

In [ ]:
# Reading data again
radar = xd.io.open_rainbow_datatree(file)

## Assign both cartesian and geographic coords

In [ ]:
radar = radar.xradar.georeference().xradar.georeference(geo=True)
display(radar["sweep_0"])

In [ ]:
del radar

## Assign to Dataset

In [ ]:
# Reading data again
radar = xd.io.open_rainbow_datatree(file)

In [ ]:
ds = radar["sweep_0"].to_dataset()

In [ ]:
display(ds)

In [ ]:
# Attach both types of coords
ds = ds.xradar.georeference().xradar.georeference(geo=True)
display(ds)

In [ ]:
ds = ds.drop(labels=["x", "y", "z", "lat", "lon", "alt"])

## Assign to DataArray

In [ ]:
reflectivity = ds["DBZH"]

In [ ]:
reflectivity = reflectivity.xradar.georeference().xradar.georeference(geo=True)
display(reflectivity)

## Plot

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5.5))

# Cartesian plot with x, y in km
reflectivity.plot(
    x="x",
    y="y",
    add_colorbar=False,
    vmin=-10,
    vmax=60,
    cmap="NWSRef",
    ax=ax[0],
)
ax[0].set_title("Reflectivity (Cartesian)")
ax[0].xaxis.set_major_formatter(ticker.FuncFormatter(lambda val, pos: f"{val/1e3:.0f}"))
ax[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda val, pos: f"{val/1e3:.0f}"))
ax[0].set_xlabel("x (km)")
ax[0].set_ylabel("y (km)")

# Geographic plot with lon, lat
pl = reflectivity.plot(
    x="lon",
    y="lat",
    add_colorbar=False,
    vmin=-10,
    vmax=60,
    cmap="NWSRef",
    ax=ax[1],
)
ax[1].set_title("Reflectivity (Geographic)")
ax[1].set_xlabel("Longitude")
ax[1].set_ylabel("Latitude")

# Add a centered colorbar below the plots
cbar = fig.colorbar(pl, ax=ax, orientation="horizontal", fraction=0.05, pad=-0.25)
cbar.set_label("Reflectivity (dBZ)")

for ax in ax.flat:
    ax.minorticks_on()
    ax.grid(ls="--", lw=0.5)

plt.tight_layout()
plt.show()

These transformations make radar data more versatile for analysis and visualization. You can apply these enhancements to entire DataTree structures or individual Dataset and DataArray objects, making xradar a flexible tool for radar data processing.